In [1]:
import sys
import os as os
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import copy
import keras as keras
from keras.layers import Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
import time
sys.path.append(os.path.abspath('squeezeNet'))
from squeezenet import *



Using Theano backend.


In [2]:
## this code creates the ground truth images, and the corresponding heatMaps, aquired by hand labelling
## of the training data. 
## the input to my net will be an image of 720x1080x3
## the output is a heatMap of 16x27

projectPath = 'C:\\Users\\David\\Desktop\\GitFolder\\project1'
dataPath = 'C:\\Users\\David\\Desktop\\Fish\\train\\train'
scorePath = os.path.join('C:\\Users\\David\\Desktop\\Fish', 'train', 'binaryScoreMaps')

# run through binaryScoreMap folders and count how many are there
fishTypes = os.listdir(scorePath)
print "fishTypes : " + fishTypes[0]
numOfImages = 0
for fishIdx in np.arange(1): #len(fishTypes)
    fishPath = os.path.join(scorePath, fishTypes[fishIdx])
    # filter all non jpg files\folders
    imageNames = [fn for fn in os.listdir(fishPath) if fn.endswith("jpg")]
    numOfImages = numOfImages + len(imageNames)
    
print 'number of score maps : ', numOfImages
h_score = 16 #720
h_img = 720
w_score = 27 #1080
w_img = 1080

# create an array of all scoreMaps & all images
scoreMaps = np.zeros((h_score,w_score,numOfImages))
images = np.zeros((numOfImages,3, h_img, w_img), dtype=np.float32)

# run through the data, and save all of it within an array
counter = 0
for fishIdx in np.arange(1): #len(fishTypes)
    fishPath = os.path.join(scorePath, fishTypes[fishIdx])
    # filter all non jpg files\folders
    scoreMapNames = [fn for fn in os.listdir(fishPath) if fn.endswith("jpg")]
    # run through the images and perform 2 things:
    # 1. resize the score images to be of size h_score x w_score
    # 2. perform all required data manipulation on the images themselfs..
    # save all...
    print 'fishType number : ' + np.str(fishIdx) + '/' + np.str(len(fishTypes))
    for imageIdx in np.arange(len(scoreMapNames)):
        # print the number of image
        #print 'image number : ' + np.str(counter) + '/' + np.str(numOfImages)
        # everything about the image
        imagePath = os.path.join(dataPath, fishTypes[fishIdx], scoreMapNames[imageIdx])
        img = misc.imread(imagePath).astype(np.float32)
        img = misc.imresize(img, (720, 1080)).astype(np.float32)
        #plt.imshow(img)        
        aux = copy.copy(img)
        img[:, :, 0] = aux[:, :, 2]
        img[:, :, 2] = aux[:, :, 0]
        # Remove train image mean
        img[:, :, 0] -= 104.006
        img[:, :, 1] -= 116.669
        img[:, :, 2] -= 122.679
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0)
        images[counter,:,:,:] = img
        
        #everything about the scoreMap
        scoreMapPath = os.path.join(fishPath, scoreMapNames[imageIdx])
        scoreMap = misc.imread(scoreMapPath, mode='L').astype(np.float32)
        scoreMap = misc.imresize(scoreMap, (h_score,w_score)).astype(np.bool)
        scoreMaps[:,:, counter] = scoreMap
        counter += 1
        

fishTypes : ALB
number of score maps :  1719
fishType number : 0/7


KeyboardInterrupt: 

In [ ]:
# save the array
import h5py
#np.save("C:\\Users\\David\\Desktop\\Fish\\train\\myFile", images)
#np.save("C:\\Users\\David\\Desktop\\Fish\\train\\myGroundTruth", scoreMaps)
#images = np.load("C:\\Users\\David\\Desktop\\Fish\\train\\myFile.npy")
#scoreMaps = np.load("C:\\Users\\David\\Desktop\\Fish\\train\\myGroundTruth.npy")

In [ ]:
# try training the network
model = get_squeezenet(2,720,1080, dim_ordering='th')
model.compile(loss="binary_crossentropy", optimizer="adam")
model.load_weights(os.path.abspath('squeezeNet/model/squeezenet_weights_th_dim_ordering_th_kernels.h5'), by_name=True)
lay = np.asarray(model.layers)

allW = np.asarray(model.get_weights())
#print 'shape : ', allW[50].shape
#model.summary()
extendedScoreMaps = np.zeros((2,scoreMaps.shape[0],scoreMaps.shape[1],scoreMaps.shape[2]))
extendedScoreMaps[0,:,:,:] = 1*np.expand_dims(scoreMaps, axis=0).astype(np.bool)
theNumberOne = np.ones((1,1,1,1))
extendedScoreMaps[1,:,:,:] = np.abs(extendedScoreMaps[0,:,:,:]-theNumberOne)
#print extendedScoreMaps.shape

transposedScoreMaps = np.transpose(extendedScoreMaps, (3, 0, 1,2))
print 'ready to start training...'

In [ ]:

partialPath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(os.path.join('C:\Users\David\Desktop\Fish\kerasModel',partialPath), monitor='val_acc', verbose=1, save_best_only=True,save_weights_only = False, mode='max')
callbacks_list = [checkpoint]
for epochNum in np.arange(4):
    model.fit(x=images, y=transposedScoreMaps, batch_size=10, nb_epoch=epochNum+1, verbose=1, callbacks=None, validation_split=0.1, shuffle=True, class_weight=np.array([1.5, 0.5]), sample_weight=None, initial_epoch=epochNum)
    model.save('C:\Users\David\Desktop\Fish\kerasModel\SecondModel' + np.str(epochNum) + '.h5')

In [ ]:
model.save('C:\Users\David\Desktop\Fish\kerasModel\FirstModel3.h5')





In [ ]:
partialPath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(os.path.join('C:\Users\David\Desktop\Fish\kerasModel',partialPath), monitor='val_acc', verbose=1, save_best_only=True,save_weights_only = False, mode='max')
callbacks_list = [checkpoint]
model.fit(x=images, y=transposedScoreMaps, batch_size=10, nb_epoch=3, verbose=1, callbacks=None, validation_split=0.1, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=2)

In [ ]:
model.save('C:\Users\David\Desktop\Fish\kerasModel\FirstModel4.h5')

In [ ]:
partialPath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(os.path.join('C:\Users\David\Desktop\Fish\kerasModel',partialPath), monitor='val_acc', verbose=1, save_best_only=True,save_weights_only = False, mode='max')
callbacks_list = [checkpoint]
model.fit(x=images, y=transposedScoreMaps, batch_size=10, nb_epoch=4, verbose=1, callbacks=None, validation_split=0.1, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=3)

In [ ]:
model.save('C:\Users\David\Desktop\Fish\kerasModel\FirstModel5.h5')